# Running ML Model Containers and APIs for Model Inference

Building and running Machine Learning model containers is a powerful approach to deploying ML models, allowing for easy scaling, portability, and version control. This notebook will guide you through:
- Building and running a Docker container for an ML model
- Use FastAPI to make the ML model accessible through a REST API endpoint to get predictions by sending data to the model

### Step 1: Setting Up Your Project
First, create a new directory for the project and navigate into it. 

```bash
mkdir ml_model_container
cd ml_model_container
```

(Note: `ml_model_container` can be found within this notebook's folder for easy reference.)

Inside this directory, create three files:
- `main.py` (for our FastAPI app) 
- `Dockerfile` (to define our Docker container)
- `requirements.txt` file (to list our Python dependencies)

### Step 2: Creating the ML Model

For this example, we will use a simple linear regression model using scikit-learn. Our model will predict a value based on a single feature. This step is simplified for demonstration purposes.

In `main.py`, add:

```python
from fastapi import FastAPI
from sklearn.linear_model import LinearRegression
import numpy as np

app = FastAPI()

# Simple model training
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])
model = LinearRegression().fit(X, y)

# Exposing an API for inference
@app.get("/predict/")
def predict(value: float):
    prediction = model.predict(np.array([[value]]))
    return {"prediction": prediction[0]}

```

### Step 3: Setting Up FastAPI

The `FastAPI` setup is already included in the `main.py` code. FastAPI will serve our ML model's predictions through a simple API.

### Step 4: Creating the Dockerfile

The Dockerfile defines the environment and instructions for running your application in a Docker container.

In `Dockerfile`, add:
```Docker
# Use an official Python runtime as a parent image
FROM python:3.8-slim

# Set the working directory in the container
WORKDIR /usr/src/app

# Copy the current directory contents into the container at /usr/src/app
COPY . .

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Define environment variable
ENV MODEL_NAME MyModel

# Run main.py when the container launches
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]
```


### Step 5: Defining Requirements
In `requirements.txt`, list all the required packages:

```tex
fastapi
uvicorn
scikit-learn
numpy
```

### Step 6: Building and Running Your Container
Navigate to your project directory in a terminal. Build your Docker container with:
```bash
docker build -t ml_model_container .
```
<img src="./imgs/docker_build.png" alt="drawing" width="8000"/>

After the build completes, run your container:
```bash
docker run -p 8000:80 ml_model_container
```
<img src="./imgs/docker_run.png" alt="drawing" width="650"/>

This command maps port 8000 on your host to port 80 on the container, making your API accessible at `http://localhost:8000`

### Step 7: Testing Your API

With your container running, you can test the prediction endpoint. 

#### Testing API using the browser

Open a web browser or use a tool like curl to access:
http://localhost:8000/predict/?value=6

<img src="./imgs/inference.png" alt="drawing" width="450"/>

#### Testing API using Python

To test your API endpoint from a Python script, you can use the `requests` library, which is a popular library used for making HTTP requests. 
```bash
pip install requests
```

In [1]:
import requests

# URL of the FastAPI app running in Docker
url = 'http://localhost:8000/predict/'

# Query parameter - value you want to predict
params = {
    'value': 6  # You can change this to any number to test different predictions
}

# Sending a GET request to the API
response = requests.get(url, params=params)

# Checking if the request was successful
if response.status_code == 200:
    # Extracting prediction from the response
    data = response.json()
    print(f"Prediction for input {params['value']}: {data['prediction']}")
else:
    print("Failed to fetch prediction", response.status_code)


Prediction for input 6: [12.000000000000002]


#### Testing API using Postman

Postman is a popular tool for API testing that allows for sending HTTP requests to your server and viewing the responses in a user-friendly interface. Follow these steps to set up Postman to test the prediction endpoint of your FastAPI application running inside a Docker container.

<img src="./imgs/postman.png" alt="drawing" width="1200"/>

##### Step 1: Install Postman

If you don't have Postman installed, download it from the [official Postman website](https://www.postman.com/downloads/). Follow the installation instructions for your operating system.

##### Step 2: Launch Postman

Open Postman after installing. You'll see an interface where you can create new requests and organize them into collections.

##### Step 3: Create a New Request

1. **Start a New Request**: Click on the "New" button or the "+" tab to create a new request.
2. **Set the Request Type**: Use the dropdown menu to select "GET" as this is the method our API endpoint uses.
3. **Enter the Request URL**: The URL should be `http://localhost:8000/predict/`, assuming your Docker setup maps port 80 inside the container to port 8000 on your host.

##### Step 4: Configure Query Parameters

1. **Add Query Parameters**: Navigate to the "Params" section below the URL bar.
2. **Key-Value Input**: Add one key-value pair:
   - **Key**: `value`
   - **Value**: Enter a number, such as `6`, for testing.

##### Step 5: Send the Request

- Click the "Send" button to make the request. Postman will send the request and display the response.

##### Step 6: View the Response

- **Response Output**: The response will appear in the lower "Body" section of the Postman interface, typically showing JSON data like `{"prediction": [12.000000000000002]}`.
- **Status and Performance**: Observe the status (e.g., 200 OK) and the response time, which are useful for performance analysis.
